In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import pairwise_kernels
import matplotlib.pyplot as plt
from tqdm import tqdm

/opt/conda/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Model

In [2]:
B = pd.read_parquet("../data/B_HKML.parquet")
all_epochs = B.date.unique()
batch_size = 25
epochs = all_epochs[:batch_size]
X_df = pd.read_parquet("../data/X_HKML.parquet")
X_train = X_df[X_df["date"].isin(epochs)].drop("date", axis=1).to_numpy()

In [3]:
i = B.shape[1] - 1
j = 0
for epoch in epochs:
    B_np = B[B["date"] == epoch].drop("date", axis=1).to_numpy().T
    j += B_np.shape[1]

Batch_B = np.zeros((i, j))

j = 0
for epoch in epochs:
    B_np = B[B["date"] == epoch].drop("date", axis=1).to_numpy().T
    dj = B_np.shape[1]
    Batch_B[:, j : j + dj] = B_np

    j += dj
B_train = Batch_B

# Gaussianize Target

In [4]:
import sys

sys.path.insert(1, "../../feature-engineering/src")
from class_ import Data

y_df = pd.read_parquet("../data/Y_HKML.parquet")

In [5]:
y_date = pd.concat([X_df["date"], y_df], axis=1)
data = Data(f_matrix=y_date, b_matrix=B)
data.gaussianize()
data.standardize()
y_date = data.f_matrix
y_df = y_date.iloc[:, 1]
y_train = y_df.iloc[: len(X_train)].to_numpy().ravel()

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

# compute coefficients

In [6]:
metric = "rbf"

In [7]:
# Values of C and D to be fine tuned to have good mse out of sample and good orthogonality out of sample?
C = 3.0
D = 0

K = pairwise_kernels(X_train, metric=metric)
BBK = B_train.T @ B_train @ K
M = K + C * np.identity(K.shape[0]) + D * BBK
dual_coef = np.linalg.solve(M, y_train)

# Estimate

In [8]:
future_moons = 50
y_hat_df = pd.DataFrame()
y_hat_df["date"] = X_df[
    X_df["date"].isin(all_epochs[batch_size : batch_size + future_moons])
].date
y_hat_df["y_hat"] = np.nan
y_hat_df["y_test"] = np.nan

In [ ]:
for moon in tqdm(range(future_moons)):
    epochs = all_epochs[0 : batch_size + moon + 1]

    j = 0
    for epoch in epochs:
        B_np = B[B["date"] == epoch].drop("date", axis=1).to_numpy().T
        j += B_np.shape[1]

    Batch_B = np.zeros((i, j))

    j = 0
    for epoch in epochs:
        B_np = B[B["date"] == epoch].drop("date", axis=1).to_numpy().T
        dj = B_np.shape[1]
        Batch_B[:, j : j + dj] = B_np

        j += dj
    B_test = Batch_B

    X_test = X_df[X_df["date"].isin(epochs)].drop("date", axis=1).to_numpy()

    y_test = y_df.iloc[: len(X_test)].to_numpy().ravel()

    K_hat = pairwise_kernels(X_test, X_train, metric=metric)

    y_hat = K_hat @ dual_coef

    exp_vec = B_test @ y_hat
    exp = np.dot(exp_vec, exp_vec)

    # print('Factor Exposure')
    # print(exp)
    # leastsquare = np.dot(y_test - y_hat, y_test - y_hat)
    # print('Least Square Error')
    # print(leastsquare)
    # ridge = K_hat @ dual_coef
    # print('Regularization Loss')
    # print(np.dot(ridge, ridge))

    y_hat_df.loc[y_hat_df["date"] == epochs[-1], "y_hat"] = y_hat[-dj:]
    y_hat_df.loc[y_hat_df["date"] == epochs[-1], "y_test"] = y_test[-dj:]

 78%|███████▊  | 39/50 [06:36<02:33, 13.93s/it]

In [ ]:
def spear(x):
    pear = x.corr(numeric_only=True)
    pear = pear.iloc[0, 1]
    spear = x.corr(numeric_only=True, method="spearman")
    spear = spear.iloc[0, 1]
    return spear

In [ ]:
metrics_history = y_hat_df.groupby("date").apply(lambda x: spear(x))

In [ ]:
metrics_history.plot(grid="on")
plt.axhline(y=metrics_history.mean(), color="r", linestyle="-")
plt.title(f"Mean: {metrics_history.mean()}")
plt.show()